In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

import json
from glob import glob
from pathlib import Path
import os


In [2]:
list(Path("../").glob("./*"))

[WindowsPath('../.git'),
 WindowsPath('../.gitignore'),
 WindowsPath('../ALL'),
 WindowsPath('../data'),
 WindowsPath('../ENG'),
 WindowsPath('../HIN'),
 WindowsPath('../IBEN'),
 WindowsPath('../LICENSE'),
 WindowsPath('../notebooks'),
 WindowsPath('../README.md'),
 WindowsPath('../src')]

In [3]:
langs = ["./ENG","./HIN","./IBEN","./ALL"]
filepaths = sum([
    [Path(v) for v in Path("../",lang).glob("./**/output/**/*_results.json")]
    for lang in langs
], [])
filepaths

[WindowsPath('../ENG/Sub-task A/output/bert-base-cased/dev_results.json'),
 WindowsPath('../ENG/Sub-task A/output/bert-base-cased/train_results.json'),
 WindowsPath('../ENG/Sub-task A/output/bert-base-uncased/dev_results.json'),
 WindowsPath('../ENG/Sub-task A/output/bert-base-uncased/train_results.json'),
 WindowsPath('../ENG/Sub-task A/output/xlm-roberta-base/dev_results.json'),
 WindowsPath('../ENG/Sub-task A/output/xlm-roberta-base/train_results.json'),
 WindowsPath('../ENG/Sub-task B/output/bert-base-cased/dev_results.json'),
 WindowsPath('../ENG/Sub-task B/output/bert-base-cased/train_results.json'),
 WindowsPath('../ENG/Sub-task B/output/bert-base-uncased/dev_results.json'),
 WindowsPath('../ENG/Sub-task B/output/bert-base-uncased/train_results.json'),
 WindowsPath('../ENG/Sub-task B/output/xlm-roberta-base/dev_results.json'),
 WindowsPath('../ENG/Sub-task B/output/xlm-roberta-base/train_results.json'),
 WindowsPath('../ENG/Sub-task C/output/bert-base-cased/dev_results.json'),
 

In [4]:
def get_filestats(filepath):
    stats = []
    with open(filepath) as fp:
        for line in fp:
            stats.append(json.loads(line))
    return stats
stats = get_filestats(filepaths[0])
len(stats)

67

In [5]:
filepaths[0].parts[1:]

('ENG', 'Sub-task A', 'output', 'bert-base-cased', 'dev_results.json')

In [6]:
TASK_MAPPING={
    "Sub-task A": "A",
    "Sub-task B": "B",
    "Sub-task C": "C",
}

MODEL_DETAILS = {
    ("ENG", "A", "bert-base-cased"): ("1"),
    ("ENG", "B", "bert-base-cased"): ("1"),
    ("ENG", "C", "bert-base-cased"): ("3"),

    ("ENG", "A", "bert-base-uncased"): ("2"),
    ("ENG", "B", "bert-base-uncased"): ("2"),
    ("ENG", "C", "bert-base-uncased"): ("4"),

    ("ENG", "A", "xlm-roberta-base"): ("5"),
    ("ENG", "B", "xlm-roberta-base"): ("5"),
    ("ENG", "C", "xlm-roberta-base"): ("6"),

    ("ENG", "A", "bert-base-multilingual-uncased"): ("7"),
    ("ENG", "B", "bert-base-multilingual-uncased"): ("8"),

    ("HIN", "A", "bert-base-multilingual-uncased"): ("1"),
    ("HIN", "B", "bert-base-multilingual-uncased"): ("1"),
    ("HIN", "C", "bert-base-multilingual-uncased"): ("3"),

    ("HIN", "A", "xlm-roberta-base"): ("2"),
    ("HIN", "B", "xlm-roberta-base"): ("2"),
    ("HIN", "C", "xlm-roberta-base"): ("4"),

    ("HIN", "A", "bert-base-multilingual-uncased"): ("5"),
    ("HIN", "B", "bert-base-multilingual-uncased"): ("6"),

    ("IBEN", "A", "bert-base-multilingual-uncased"): ("1"),
    ("IBEN", "B", "bert-base-multilingual-uncased"): ("1"),
    ("IBEN", "C", "bert-base-multilingual-uncased"): ("3"),

    ("IBEN", "A", "xlm-roberta-base"): ("2"),
    ("IBEN", "B", "xlm-roberta-base"): ("2"),
    ("IBEN", "C", "xlm-roberta-base"): ("4"),

    ("IBEN", "A", "bert-base-multilingual-uncased"): ("5"),
    ("IBEN", "B", "bert-base-multilingual-uncased"): ("6"),
    
}


In [7]:
def get_model_C_stats_old(df, lang, model, task, split, run_id):
    lblsA = ['OAG','NAG','CAG']
    lblsB = ['GEN','NGEN']
    Albls = {i:l for i,l in enumerate(lblsA)}
    Blbls = {i:l for i,l in enumerate(lblsB)}
    for i in lblsA:
        df[i+'_probs'] = df[i+'-'+lblsB[0]+'_probs']+df[i+'-'+lblsB[1]+'_probs']  
    for i in lblsB:
        df[i+'_probs'] = df[lblsA[0]+'-'+i+'_probs']+df[lblsA[1]+'-'+i+'_probs']+df[lblsA[2]+'-'+i+'_probs']  
    a = pd.DataFrame({'val':np.argmax( [df[lblsA[0]+'_probs'] , df[lblsA[1]+'_probs'], df[lblsA[2]+'_probs']],axis=0)})
    b = pd.DataFrame({'val':np.argmax([df[lblsB[0]+'_probs'], df[lblsB[1]+'_probs']],axis=0)})
    df["A_preds"] = a['val'].map(Albls)
    df["B_preds"] = b['val'].map(Blbls)
    df["Sub-task C_mpreds"] = df["A_preds"]+'-'+df["B_preds"]

    tmp = {
        0 : "Sub-task C_preds",
        1 : "Sub-task C_mpreds"
    } 

    tmp1 = {
        0 : "C",
        1 : "M"
    }
    
    for j in range(2):
        for i, task in enumerate(["A", "B"]):
            report = classification_report(
                df["label"].str.split("-", expand=True)[i],
                df[tmp[j]].str.split("-", expand=True)[i],
                output_dict=True
            )
            final_stat = {
                "macro avg/f1-score": report["macro avg"]["f1-score"],
                "weighted avg/f1-score": report["weighted avg"]["f1-score"]
            }
            final_stat.update(dict(
                lang=lang, model=f"{model} ({tmp1[j]})", task=task, split=split, run_id=f"{run_id} ({tmp1[j]})"
            ))
            yield final_stat
            
def add_marginalized_predictions_for_subtask_C(df):
    column_names=df.filter(like='_probs', axis=1).columns.tolist()
    listA=list(set([
        x.split('-')[0] 
        for x in column_names]))
    listB=list(set([
        x[x.find('-')+1:x.find('_')] 
        for x in column_names]))
    for label in listA:
        filter_str=r'^{}-'.format(label)
        df[label+'_probs']=df.filter(regex=filter_str, axis=1).sum(axis=1)
    for label in listB:
        filter_str="-{}".format(label)
        df[label+'_probs']=df.filter(regex=filter_str, axis=1).sum(axis=1)
    prob_listA=[x+'_probs' for x in listA]
    prob_listB=[x+'_probs' for x in listB]
    
    df['Sub-task B_preds']=df[prob_listB].idxmax(axis=1).apply(
        lambda x: pd.Series(str(x).split('_'))[0])
    df['Sub-task A_preds']=df[prob_listA].idxmax(axis=1).apply(
        lambda x: pd.Series(str(x).split('_'))[0])
    df['Sub-task C_mpreds'] = df['Sub-task A_preds'].str.cat(df['Sub-task B_preds'], "-")
    return df
            
            
def get_model_C_stats(df, lang, model, task, split, run_id):
    df = add_marginalized_predictions_for_subtask_C(df)
    tmp = {
        0 : "Sub-task C_preds",
        1 : "Sub-task C_mpreds"
    } 

    tmp1 = {
        0 : "C",
        1 : "M"
    }
    
    for j in range(2):
        for i, task in enumerate(["A", "B"]):
            report = classification_report(
                df["label"].str.split("-", expand=True)[i],
                df[tmp[j]].str.split("-", expand=True)[i],
                output_dict=True
            )
            final_stat = {
                "macro avg/f1-score": report["macro avg"]["f1-score"],
                "weighted avg/f1-score": report["weighted avg"]["f1-score"]
            }
            final_stat.update(dict(
                lang=lang, model=f"{model} ({tmp1[j]})", task=task, split=split, run_id=f"{run_id} ({tmp1[j]})"
            ))
            yield final_stat

def get_model_ALL_stats(df, lang, model, task, split, run_id):
    for i, l in enumerate(["ENG", "HIN", "IBEN"]):
        #run_id = ""
        #    if lang == "DE" and task == "C":
        #        continue
        #    if i+1 in run_id_tasks:
        #        run_id = base_run_id
        pred_label = "Sub-task "+task+"_preds"
        actual_label = "label"
        df_t = df.loc[df["id"].str.split('-',expand=True)[0]==l].copy()
        if task != "C":
            report = classification_report(
                df_t[actual_label],
                df_t[pred_label],
                output_dict=True
            )
            final_stat = {
                "macro avg/f1-score": report["macro avg"]["f1-score"],
                "weighted avg/f1-score": report["weighted avg"]["f1-score"]
            }
            final_stat.update(dict(lang=l, model=model, task=task, split=split, run_id=run_id))
            yield final_stat
        else:
            yield from get_model_C_stats(df_t, l, model, task, split, run_id)

def get_final_stats(filepath):
    lang, task, _, model, basename = filepath.parts[1:]
    stats = get_filestats(filepath)
    split = basename.split("_")[0]
    task = TASK_MAPPING[task]
    default_model_details = ("", )
    run_id, = MODEL_DETAILS.get((lang, task, model), default_model_details)
    
    base_path = filepath.parent
    final_path = base_path / filepath.name.replace("_results.json", ".tsv")
    print(filepath, lang, model, task, split, run_id)
    
    df = pd.read_csv(final_path, sep="\t")
    if lang == "ALL":
        run_id = 9
        return [
            final_stat
            for final_stat in get_model_ALL_stats(
                df, lang=lang, model=f"{model} (ALL)", task=task, split=split, run_id=run_id
            )
        ]
    if task == "C":
        #print("C\n")
        return [
            final_stat
            for final_stat in get_model_C_stats(
                df, lang=lang, model=f"{model} (C)", task=task, split=split, run_id=run_id
            )
        ]
    final_stat = stats[-1]
    final_stat.update(dict(lang=lang, model=model, task=task, split=split, run_id=run_id))
    #print(lang,model,task)
    return [final_stat]


In [12]:
all_stats = %time sum((get_final_stats(filepath) for filepath in filepaths), [])
len(all_stats)
all_stats

..\ENG\Sub-task A\output\bert-base-cased\dev_results.json ENG bert-base-cased A dev 1
..\ENG\Sub-task A\output\bert-base-cased\train_results.json ENG bert-base-cased A train 1
..\ENG\Sub-task A\output\bert-base-uncased\dev_results.json ENG bert-base-uncased A dev 2
..\ENG\Sub-task A\output\bert-base-uncased\train_results.json ENG bert-base-uncased A train 2
..\ENG\Sub-task A\output\xlm-roberta-base\dev_results.json ENG xlm-roberta-base A dev 5
..\ENG\Sub-task A\output\xlm-roberta-base\train_results.json ENG xlm-roberta-base A train 5
..\ENG\Sub-task B\output\bert-base-cased\dev_results.json ENG bert-base-cased B dev 1
..\ENG\Sub-task B\output\bert-base-cased\train_results.json ENG bert-base-cased B train 1
..\ENG\Sub-task B\output\bert-base-uncased\dev_results.json ENG bert-base-uncased B dev 2
..\ENG\Sub-task B\output\bert-base-uncased\train_results.json ENG bert-base-uncased B train 2
..\ENG\Sub-task B\output\xlm-roberta-base\dev_results.json ENG xlm-roberta-base B dev 5
..\ENG\Sub-t

[{'eval_loss': 0.747566761339412,
  'OAG/precision': 0.5679012345679012,
  'OAG/recall': 0.40707964601769914,
  'OAG/f1-score': 0.4742268041237113,
  'OAG/support': 113,
  'NAG/precision': 0.8841463414634146,
  'NAG/recall': 0.8672248803827751,
  'NAG/f1-score': 0.8756038647342996,
  'NAG/support': 836,
  'CAG/precision': 0.2909090909090909,
  'CAG/recall': 0.41025641025641024,
  'CAG/f1-score': 0.3404255319148936,
  'CAG/support': 117,
  'accuracy': 0.7682926829268293,
  'macro avg/precision': 0.5809855556468023,
  'macro avg/recall': 0.5615203122189615,
  'macro avg/f1-score': 0.5634187335909683,
  'macro avg/support': 1066,
  'weighted avg/precision': 0.785511767923031,
  'weighted avg/recall': 0.7682926829268293,
  'weighted avg/f1-score': 0.7743173048948372,
  'weighted avg/support': 1066,
  'global_step': 335,
  'loss': 0.08183887004852294,
  'lang': 'ENG',
  'model': 'bert-base-cased',
  'task': 'A',
  'split': 'dev',
  'run_id': '1'},
 {'eval_loss': 0.05159350417887987,
  'OAG/

In [13]:
df_map = {
    k: pd.DataFrame([stats for stats in all_stats if stats["task"] == k])
    for k in ["A", "B",]
}
df_map["A"].head()

,eval_loss,OAG/precision,OAG/recall,OAG/f1-score,OAG/support,NAG/precision,NAG/recall,NAG/f1-score,NAG/support,CAG/precision,...,weighted avg/recall,weighted avg/f1-score,weighted avg/support,global_step,loss,lang,model,task,split,run_id
0,0.747567,0.567901,0.407080,0.474227,113.0,0.884146,0.867225,0.875604,836.0,0.290909,...,0.768293,0.774317,1066.0,335.0,0.081839,ENG,bert-base-cased,A,dev,1
1,0.051594,0.949886,0.958621,0.954233,435.0,0.996443,0.996148,0.996296,3375.0,0.951111,...,0.986864,0.986866,4263.0,335.0,0.081839,ENG,bert-base-cased,A,train,1
2,0.692995,0.666667,0.460177,0.544503,113.0,0.878394,0.889952,0.884135,836.0,0.276596,...,0.783302,0.784276,1066.0,335.0,0.162729,ENG,bert-base-uncased,A,dev,2
3,0.082985,0.908297,0.956322,0.931691,435.0,0.992604,0.994074,0.993338,3375.0,0.948235,...,0.979123,0.979042,4263.0,335.0,0.162729,ENG,bert-base-uncased,A,train,2
4,0.598518,0.428571,0.424779,0.426667,113.0,0.878363,0.898325,0.888232,836.0,0.303030,...,0.777674,0.772303,1066.0,1320.0,0.390615,ENG,xlm-roberta-base,A,dev,5


In [14]:
with pd.option_context("precision", 3):
    for k in ["A", "B"]:
        for l in ["ENG", "HIN", "IBEN"]:
            print(f"subtask={k}, lang={l}")
            df_t = df_map[k][df_map[k].lang==l][["lang", "task", "model", "run_id", "split", "macro avg/f1-score", "weighted avg/f1-score"]].rename(columns={
                "macro avg/f1-score": "macro", "weighted avg/f1-score": "weighted"
            }).pivot_table(index=["lang", "task", "model", "run_id"], columns=["split"])
            df_t["rank"] = df_t[("weighted", "dev")].rank(ascending=False).astype(int)
            display(df_t.sort_values("rank"))

subtask=A, lang=ENG


macro         \
split                                                        dev  train   
lang task model                                    run_id                 
ENG  A    bert-base-multilingual-uncased (ALL)     9       0.611  0.903   
          bert-base-uncased (C) (C)                4 (C)   0.596  0.902   
          bert-base-uncased (C) (M)                4 (M)   0.595  0.900   
          bert-base-cased (C) (C)                  3 (C)   0.571  0.912   
          bert-base-uncased                        2       0.577  0.948   
          bert-base-cased (C) (M)                  3 (M)   0.568  0.908   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.555  0.865   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.550  0.871   
          bert-base-cased                          1       0.563  0.966   
          xlm-roberta-base                         5       0.531  0.676   
          xlm-roberta-base (C) (C)                 6 (C)   0.515  0.640   
          xlm-roberta-base (ALL)                   9       0.512  0.610   
          xlm-roberta-base (C) (M)                 6 (M)   0.518  0.634   

                                                          weighted        rank  
split                                                          dev  train       
lang task model                                    run_id                       
ENG  A    bert-base-multilingual-uncased (ALL)     9         0.798  0.957    1  
          bert-base-uncased (C) (C)                4 (C)     0.795  0.956    2  
          bert-base-uncased (C) (M)                4 (M)     0.795  0.956    3  
          bert-base-cased (C) (C)                  3 (C)     0.786  0.961    4  
          bert-base-uncased                        2         0.784  0.979    5  
          bert-base-cased (C) (M)                  3 (M)     0.782  0.960    6  
          bert-base-multilingual-uncased (ALL) (M) 9 (M)     0.780  0.939    7  
          bert-base-multilingual-uncased (ALL) (C) 9 (C)     0.778  0.941    8  
          bert-base-cased                          1         0.774  0.987    9  
          xlm-roberta-base                         5         0.772  0.862   10  
          xlm-roberta-base (C) (C)                 6 (C)     0.762  0.835   11  
          xlm-roberta-base (ALL)                   9         0.762  0.823   12  
          xlm-roberta-base (C) (M)                 6 (M)     0.761  0.830   13

subtask=A, lang=HIN


macro         \
split                                                        dev  train   
lang task model                                    run_id                 
HIN  A    bert-base-multilingual-uncased           5       0.637  0.846   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.628  0.903   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.626  0.899   
          bert-base-multilingual-uncased (ALL)     9       0.626  0.939   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.616  0.849   
          bert-base-multilingual-uncased (C) (M)   3 (M)   0.611  0.848   
          xlm-roberta-base (ALL)                   9       0.598  0.698   
          xlm-roberta-base                         2       0.394  0.388   
          xlm-roberta-base (C) (C)                 4 (C)   0.245  0.240   
          xlm-roberta-base (C) (M)                 4 (M)   0.245  0.240   

                                                          weighted        rank  
split                                                          dev  train       
lang task model                                    run_id                       
HIN  A    bert-base-multilingual-uncased           5         0.708  0.881    1  
          bert-base-multilingual-uncased (ALL) (C) 9 (C)     0.696  0.924    2  
          bert-base-multilingual-uncased (ALL) (M) 9 (M)     0.695  0.921    3  
          bert-base-multilingual-uncased (ALL)     9         0.694  0.952    4  
          bert-base-multilingual-uncased (C) (C)   3 (C)     0.688  0.884    5  
          bert-base-multilingual-uncased (C) (M)   3 (M)     0.684  0.884    6  
          xlm-roberta-base (ALL)                   9         0.672  0.753    7  
          xlm-roberta-base                         2         0.527  0.509    8  
          xlm-roberta-base (C) (C)                 4 (C)     0.426  0.406    9  
          xlm-roberta-base (C) (M)                 4 (M)     0.426  0.406    9

subtask=A, lang=IBEN


...

macro         \
split                                                        dev  train   
lang task model                                    run_id                 
ENG  B    bert-base-uncased (C) (M)                4 (M)   0.757  0.920   
          xlm-roberta-base (ALL)                   9       0.765  0.878   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.760  0.939   
          bert-base-uncased (C) (C)                4 (C)   0.734  0.914   
          bert-base-cased (C) (C)                  3 (C)   0.729  0.931   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.752  0.936   
          bert-base-cased (C) (M)                  3 (M)   0.727  0.935   
          bert-base-uncased                        2       0.737  0.991   
          bert-base-multilingual-uncased (ALL)     9       0.751  0.987   
          xlm-roberta-base                         5       0.734  0.915   
          xlm-roberta-base (C) (M)                 6 (M)   0.728  0.807   
          xlm-roberta-base (C) (C)                 6 (C)   0.711  0.813   
          bert-base-cased                          1       0.700  0.982   

                                                          weighted        rank  
split                                                          dev  train       
lang task model                                    run_id                       
ENG  B    bert-base-uncased (C) (M)                4 (M)     0.943  0.978    1  
          xlm-roberta-base (ALL)                   9         0.941  0.968    2  
          bert-base-multilingual-uncased (ALL) (M) 9 (M)     0.940  0.983    3  
          bert-base-uncased (C) (C)                4 (C)     0.939  0.977    4  
          bert-base-cased (C) (C)                  3 (C)     0.939  0.982    5  
          bert-base-multilingual-uncased (ALL) (C) 9 (C)     0.938  0.983    6  
          bert-base-cased (C) (M)                  3 (M)     0.938  0.983    7  
          bert-base-uncased                        2         0.938  0.998    8  
          bert-base-multilingual-uncased (ALL)     9         0.937  0.996    9  
          xlm-roberta-base                         5         0.936  0.978   10  
          xlm-roberta-base (C) (M)                 6 (M)     0.934  0.948   11  
          xlm-roberta-base (C) (C)                 6 (C)     0.933  0.952   12  
          bert-base-cased                          1         0.929  0.995   13

subtask=B, lang=HIN


macro         \
split                                                        dev  train   
lang task model                                    run_id                 
HIN  B    bert-base-multilingual-uncased           6       0.780  0.974   
          bert-base-multilingual-uncased (ALL)     9       0.778  0.990   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.783  0.932   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.778  0.931   
          bert-base-multilingual-uncased (C) (M)   3 (M)   0.760  0.844   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.750  0.847   
          xlm-roberta-base (ALL)                   9       0.745  0.831   
          xlm-roberta-base                         2       0.459  0.455   
          xlm-roberta-base (C) (C)                 4 (C)   0.459  0.455   
          xlm-roberta-base (C) (M)                 4 (M)   0.459  0.455   

                                                          weighted        rank  
split                                                          dev  train       
lang task model                                    run_id                       
HIN  B    bert-base-multilingual-uncased           6         0.891  0.986    1  
          bert-base-multilingual-uncased (ALL)     9         0.888  0.994    2  
          bert-base-multilingual-uncased (ALL) (C) 9 (C)     0.888  0.962    3  
          bert-base-multilingual-uncased (ALL) (M) 9 (M)     0.886  0.962    4  
          bert-base-multilingual-uncased (C) (M)   3 (M)     0.882  0.916    5  
          bert-base-multilingual-uncased (C) (C)   3 (C)     0.874  0.917    6  
          xlm-roberta-base (ALL)                   9         0.870  0.909    7  
          xlm-roberta-base                         2         0.778  0.759    9  
          xlm-roberta-base (C) (C)                 4 (C)     0.778  0.759    9  
          xlm-roberta-base (C) (M)                 4 (M)     0.778  0.759    9

subtask=B, lang=IBEN


macro         \
split                                                        dev  train   
lang task model                                    run_id                 
IBEN B    bert-base-multilingual-uncased (ALL)     9       0.849  0.987   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.849  0.943   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.846  0.943   
          bert-base-multilingual-uncased           6       0.830  0.975   
          bert-base-multilingual-uncased (C) (M)   3 (M)   0.827  0.924   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.824  0.923   
          xlm-roberta-base (ALL)                   9       0.792  0.845   
          xlm-roberta-base (C) (M)                 4 (M)   0.783  0.835   
          xlm-roberta-base (C) (C)                 4 (C)   0.783  0.833   
          xlm-roberta-base                         2       0.714  0.743   

                                                          weighted        rank  
split                                                          dev  train       
lang task model                                    run_id                       
IBEN B    bert-base-multilingual-uncased (ALL)     9         0.905  0.992    1  
          bert-base-multilingual-uncased (ALL) (M) 9 (M)     0.904  0.965    2  
          bert-base-multilingual-uncased (ALL) (C) 9 (C)     0.902  0.966    3  
          bert-base-multilingual-uncased           6         0.894  0.985    4  
          bert-base-multilingual-uncased (C) (M)   3 (M)     0.892  0.954    5  
          bert-base-multilingual-uncased (C) (C)   3 (C)     0.890  0.953    6  
          xlm-roberta-base (ALL)                   9         0.873  0.908    7  
          xlm-roberta-base (C) (M)                 4 (M)     0.869  0.903    8  
          xlm-roberta-base (C) (C)                 4 (C)     0.868  0.902    9  
          xlm-roberta-base                         2         0.830  0.855   10

In [15]:
with pd.option_context("precision", 3, "max_rows", 100):
    df_t = pd.concat([df_map[k][["lang", "task", "model", "run_id", "split", "macro avg/f1-score", "weighted avg/f1-score"]].rename(columns={
            "macro avg/f1-score": "macro", "weighted avg/f1-score": "weighted"
        }) for k in ["A", "B"]])
    df_t = df_t.pivot_table(index=["task", "lang", "model", "run_id"], columns=["split"])
    display(df_t)

macro         \
split                                                        dev  train   
task lang model                                    run_id                 
A    ENG  bert-base-cased                          1       0.563  0.966   
          bert-base-cased (C) (C)                  3 (C)   0.571  0.912   
          bert-base-cased (C) (M)                  3 (M)   0.568  0.908   
          bert-base-multilingual-uncased (ALL)     9       0.611  0.903   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.550  0.871   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.555  0.865   
          bert-base-uncased                        2       0.577  0.948   
          bert-base-uncased (C) (C)                4 (C)   0.596  0.902   
          bert-base-uncased (C) (M)                4 (M)   0.595  0.900   
          xlm-roberta-base                         5       0.531  0.676   
          xlm-roberta-base (ALL)                   9       0.512  0.610   
          xlm-roberta-base (C) (C)                 6 (C)   0.515  0.640   
          xlm-roberta-base (C) (M)                 6 (M)   0.518  0.634   
     HIN  bert-base-multilingual-uncased           5       0.637  0.846   
          bert-base-multilingual-uncased (ALL)     9       0.626  0.939   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.628  0.903   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.626  0.899   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.616  0.849   
          bert-base-multilingual-uncased (C) (M)   3 (M)   0.611  0.848   
          xlm-roberta-base                         2       0.394  0.388   
          xlm-roberta-base (ALL)                   9       0.598  0.698   
          xlm-roberta-base (C) (C)                 4 (C)   0.245  0.240   
          xlm-roberta-base (C) (M)                 4 (M)   0.245  0.240   
     IBEN bert-base-multilingual-uncased           5       0.680  0.903   
          bert-base-multilingual-uncased (ALL)     9       0.698  0.933   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.686  0.893   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.683  0.893   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.684  0.860   
          bert-base-multilingual-uncased (C) (M)   3 (M)   0.686  0.856   
          xlm-roberta-base                         2       0.584  0.631   
          xlm-roberta-base (ALL)                   9       0.663  0.728   
          xlm-roberta-base (C) (C)                 4 (C)   0.692  0.757   
          xlm-roberta-base (C) (M)                 4 (M)   0.694  0.758   
B    ENG  bert-base-cased                          1       0.700  0.982   
          bert-base-cased (C) (C)                  3 (C)   0.729  0.931   
          bert-base-cased (C) (M)                  3 (M)   0.727  0.935   
          bert-base-multilingual-uncased (ALL)     9       0.751  0.987   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.752  0.936   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.760  0.939   
          bert-base-uncased                        2       0.737  0.991   
          bert-base-uncased (C) (C)                4 (C)   0.734  0.914   
          bert-base-uncased (C) (M)                4 (M)   0.757  0.920   
          xlm-roberta-base                         5       0.734  0.915   
          xlm-roberta-base (ALL)                   9       0.765  0.878   
          xlm-roberta-base (C) (C)                 6 (C)   0.711  0.813   
          xlm-roberta-base (C) (M)                 6 (M)   0.728  0.807   
     HIN  bert-base-multilingual-uncased           6       0.780  0.974   
          bert-base-multilingual-uncased (ALL)     9       0.778  0.990   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.783  0.932   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.778  0.931   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.750  0.847   
         

In [16]:
with pd.option_context("precision", 3, "max_rows", 100):
    df_t = pd.concat([df_map[k][["lang", "task", "model", "run_id", "split", "macro avg/f1-score", "weighted avg/f1-score"]].rename(columns={
            "macro avg/f1-score": "macro", "weighted avg/f1-score": "weighted"
        }) for k in ["A", "B"]])
    df_t = df_t.pivot_table(index=["lang", "task", "model", "run_id"], columns=["split"])
    display(df_t)

macro         \
split                                                        dev  train   
lang task model                                    run_id                 
ENG  A    bert-base-cased                          1       0.563  0.966   
          bert-base-cased (C) (C)                  3 (C)   0.571  0.912   
          bert-base-cased (C) (M)                  3 (M)   0.568  0.908   
          bert-base-multilingual-uncased (ALL)     9       0.611  0.903   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.550  0.871   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.555  0.865   
          bert-base-uncased                        2       0.577  0.948   
          bert-base-uncased (C) (C)                4 (C)   0.596  0.902   
          bert-base-uncased (C) (M)                4 (M)   0.595  0.900   
          xlm-roberta-base                         5       0.531  0.676   
          xlm-roberta-base (ALL)                   9       0.512  0.610   
          xlm-roberta-base (C) (C)                 6 (C)   0.515  0.640   
          xlm-roberta-base (C) (M)                 6 (M)   0.518  0.634   
     B    bert-base-cased                          1       0.700  0.982   
          bert-base-cased (C) (C)                  3 (C)   0.729  0.931   
          bert-base-cased (C) (M)                  3 (M)   0.727  0.935   
          bert-base-multilingual-uncased (ALL)     9       0.751  0.987   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.752  0.936   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.760  0.939   
          bert-base-uncased                        2       0.737  0.991   
          bert-base-uncased (C) (C)                4 (C)   0.734  0.914   
          bert-base-uncased (C) (M)                4 (M)   0.757  0.920   
          xlm-roberta-base                         5       0.734  0.915   
          xlm-roberta-base (ALL)                   9       0.765  0.878   
          xlm-roberta-base (C) (C)                 6 (C)   0.711  0.813   
          xlm-roberta-base (C) (M)                 6 (M)   0.728  0.807   
HIN  A    bert-base-multilingual-uncased           5       0.637  0.846   
          bert-base-multilingual-uncased (ALL)     9       0.626  0.939   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.628  0.903   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.626  0.899   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.616  0.849   
          bert-base-multilingual-uncased (C) (M)   3 (M)   0.611  0.848   
          xlm-roberta-base                         2       0.394  0.388   
          xlm-roberta-base (ALL)                   9       0.598  0.698   
          xlm-roberta-base (C) (C)                 4 (C)   0.245  0.240   
          xlm-roberta-base (C) (M)                 4 (M)   0.245  0.240   
     B    bert-base-multilingual-uncased           6       0.780  0.974   
          bert-base-multilingual-uncased (ALL)     9       0.778  0.990   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.783  0.932   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.778  0.931   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.750  0.847   
          bert-base-multilingual-uncased (C) (M)   3 (M)   0.760  0.844   
          xlm-roberta-base                         2       0.459  0.455   
          xlm-roberta-base (ALL)                   9       0.745  0.831   
          xlm-roberta-base (C) (C)                 4 (C)   0.459  0.455   
          xlm-roberta-base (C) (M)                 4 (M)   0.459  0.455   
IBEN A    bert-base-multilingual-uncased           5       0.680  0.903   
          bert-base-multilingual-uncased (ALL)     9       0.698  0.933   
          bert-base-multilingual-uncased (ALL) (C) 9 (C)   0.686  0.893   
          bert-base-multilingual-uncased (ALL) (M) 9 (M)   0.683  0.893   
          bert-base-multilingual-uncased (C) (C)   3 (C)   0.684  0.860   
         